In [5]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
load_dotenv()
from langdetect import detect, LangDetectException
import time
from bs4 import BeautifulSoup


In [3]:
best_books_0 = pd.read_csv('books_1.Best_Books_Ever.csv')

In [241]:
best_books_0.head()

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,01/28/13,[],2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,10/05/05,"['Georgia Peach Book Award (2007)', 'Buxtehude...",4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1


In [259]:
best_books_0.shape

(52478, 25)

In [11]:
best_books_0.isnull().sum()


bookId                         0
title                          0
series                     28982
author                         0
rating                         0
description                 1336
language                    3801
isbn                           0
genres                         0
characters                     0
bookFormat                  1473
edition                    47475
pages                       2343
publisher                   3692
publishDate                  879
firstPublishDate           21302
awards                         0
numRatings                     0
ratingsByStars                 0
likedPercent                 621
setting                        0
coverImg                     605
bbeScore                       0
bbeVotes                       0
price                      14344
bag_of_words                   0
hyphenated_url_bookshop        0
url                            0
dtype: int64

In [12]:
best_books_0.duplicated(subset='bookId').sum()

0

In [260]:
best_books_0.drop_duplicates(subset='bookId', inplace=True)

In [261]:

# duplicates = best_books[best_books.duplicated(subset='bookId', keep=False)]

# # Display the duplicated rows
# duplicates_sorted = duplicates.sort_values(by='bookId')

# # Display the sorted duplicated rows
# print(duplicates_sorted)

Empty DataFrame
Columns: [bookId, title, series, author, rating, description, language, isbn, genres, characters, bookFormat, edition, pages, publisher, publishDate, firstPublishDate, awards, numRatings, ratingsByStars, likedPercent, setting, coverImg, bbeScore, bbeVotes, price]
Index: []

[0 rows x 25 columns]


In [ ]:
best_books_0.head(20)

In [263]:
best_books_0['series'].notna()

0         True
1         True
2         True
3        False
4         True
         ...  
52473     True
52474     True
52475     True
52476    False
52477     True
Name: series, Length: 52424, dtype: bool

In [13]:
def create_bag_of_words(row):
    components = [
        row['title'],
        row['author'],
        row['awards'],
        row['setting'],
        row['series'] if pd.notna(row['series']) else '',
        row['description'] if pd.notna(row['description']) else '',
        row['language'] if pd.notna(row['language']) else ''
    ]
    return ' '.join(components)

best_books_0['bag_of_words'] = best_books_0.apply(create_bag_of_words, axis=1)

print(best_books_0)

                                            bookId  \
0                         2767052-the-hunger-games   
1      2.Harry_Potter_and_the_Order_of_the_Phoenix   
2                       2657.To_Kill_a_Mockingbird   
3                         1885.Pride_and_Prejudice   
4                                   41865.Twilight   
...                                            ...   
52419                           11492014-fractured   
52420                             11836711-anasazi   
52421                              10815662-marked   
52422                         11330278-wayward-son   
52423                 10991547-daughter-of-helaman   

                                           title                 series  \
0                               The Hunger Games    The Hunger Games #1   
1      Harry Potter and the Order of the Phoenix        Harry Potter #5   
2                          To Kill a Mockingbird  To Kill a Mockingbird   
3                            Pride and Prejudice   

In [268]:
best_books_0.to_csv('df_books_final.csv', index=False)

In [13]:
best_books = pd.read_csv('df_books_final.csv')


In [14]:
best_books.shape

(52424, 26)

In [4]:
best_books['hyphenated_url_bookshop'] = best_books['title'].str.split(" ").str.join("+")+"+"+best_books["author"].str.split(" ").str.join("+")

best_books['url'] = "https://bookshop.org/search?keywords="+best_books['hyphenated_url_bookshop']

In [5]:
best_books['wordery_url'] = "https://wordery.com/search?term="+best_books['title'].str.split(" ").str.join("+")+"+"+best_books["author"].str.split(" ").str.join("+")

In [6]:
# best_books['hyphenated_url_NYP'] = "https://borrow.nypl.org/search?query="+best_books['searc_params_NYP']+"&searchType=everything&pageSize=10"


KeyError: 'searc_params_NYP'

In [72]:
# best_books['searc_params_NYP']= best_books['title'].str.split(" ").str.join("%20")+"%20"+best_books["author"].str.split(" ").str.join("%20")

In [22]:
# best_books['url'][2]

'https://bookshop.org/search?keywords=To+Kill+a+Mockingbird+Harper+Lee'

In [90]:
# best_books.drop(columns='cover_picture', inplace=True)

In [7]:
best_books.head()

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,likedPercent,setting,coverImg,bbeScore,bbeVotes,price,bag_of_words,hyphenated_url_bookshop,url,wordery_url
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09,The Hunger Games Suzanne Collins ['Locus Award...,The+Hunger+Games+Suzanne+Collins,https://bookshop.org/search?keywords=The+Hunge...,https://wordery.com/search?term=The+Hunger+Gam...
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38,Harry Potter and the Order of the Phoenix J.K....,Harry+Potter+and+the+Order+of+the+Phoenix+J.K....,https://bookshop.org/search?keywords=Harry+Pot...,https://wordery.com/search?term=Harry+Potter+a...
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN,To Kill a Mockingbird Harper Lee ['Pulitzer Pr...,To+Kill+a+Mockingbird+Harper+Lee,https://bookshop.org/search?keywords=To+Kill+a...,https://wordery.com/search?term=To+Kill+a+Mock...
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN,"Pride and Prejudice Jane Austen, Anna Quindlen...","Pride+and+Prejudice+Jane+Austen,+Anna+Quindlen...",https://bookshop.org/search?keywords=Pride+and...,https://wordery.com/search?term=Pride+and+Prej...
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1,Twilight Stephenie Meyer ['Georgia Peach Book ...,Twilight+Stephenie+Meyer,https://bookshop.org/search?keywords=Twilight+...,https://wordery.com/search?term=Twilight+Steph...


In [55]:
# def get_cover(row):
#     url = row['hyphenated_url_NYP']
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#     }
#     print(f"Fetching URL: {url}")
#     response = requests.get(url, headers=headers, timeout=10)
#     if response.status_code == 200:
#         soup = BeautifulSoup(response.content, 'html.parser')
#         img_tag = soup.select_one('.app-cover-container img')
#         # Extract the src attribute
#         if img_tag:
#             return img_tag.get('src')

# # Apply the get_cover function and save progress periodically
# output_file = 'best_books_with_covers.csv'
# save_interval = 10  # Save progress every 10 iterations

# for i, row in best_books.iterrows():
#     best_books.at[i, 'cover_picture'] = get_cover(row)
#     print(f"Cover picture for row {i}: {best_books.at[i, 'cover_picture']}")
#     if i % save_interval == 0:
#         best_books.to_csv(output_file, index=False)
#     time.sleep(2)  # Delay between requests to avoid being blocked

# # Final save
# best_books.to_csv(output_file, index=False)

# # Check the DataFrame
# print(best_books)

Fetching URL: https://borrow.nypl.org/search?query=The%20Hunger%20Games+Suzanne%20Collins&searchType=everything&pageSize=10
Cover picture for row 0: None
Fetching URL: https://borrow.nypl.org/search?query=Harry%20Potter%20and%20the%20Order%20of%20the%20Phoenix+J.K.%20Rowling,%20Mary%20GrandPré%20(Illustrator)&searchType=everything&pageSize=10
Cover picture for row 1: None
Fetching URL: https://borrow.nypl.org/search?query=To%20Kill%20a%20Mockingbird+Harper%20Lee&searchType=everything&pageSize=10


KeyboardInterrupt: 

In [80]:
# def get_cover(row):
#     url = row['hyphenated_url_NYP']
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
#                       '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#     }
#     print(f"Fetching URL: {url}")
#     try:
#         response = requests.get(url, headers=headers, timeout=10)
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.content, 'html.parser')
#             img_tag = soup.select_one('div.card-main-content.col > div:nth-child(1) > div.col-auto > div > app-cover > img')
#             # Extract the src attribute
#             #rollup-card-0 > div > div > div.card-main-content.col > div:nth-child(1) > div.col-auto > div > app-cover > img
#             if img_tag:
#                 return img_tag.get('src')
#             else:
#                 print(f"No img tag found for URL: {url}")
#         else:
#             print(f"Failed to fetch URL: {url}, Status code: {response.status_code}")
#     except requests.exceptions.RequestException as e:
#         print(f"An error occurred: {e}")
#     return None  # Return None if any error occurs or image tag not found

# # Apply the get_cover function and save progress periodically
# output_file = 'best_books_with_covers.csv'
# save_interval = 10  # Save progress every 10 iterations

# for i, row in best_books.iterrows():
#     cover_picture = get_cover(row)
#     best_books.at[i, 'cover_picture'] = cover_picture
#     print(f"Cover picture for row {i}: {cover_picture}")
#     if i % save_interval == 0:
#         best_books.to_csv(output_file, index=False)
#     time.sleep(2)  # Delay between requests to avoid being blocked

# # Final save
# best_books.to_csv(output_file, index=False)

# # Check the DataFrame
# print(best_books)

Fetching URL: https://borrow.nypl.org/search?query=The%20Hunger%20Games%20Suzanne%20Collins&searchType=everything&pageSize=10
No img tag found for URL: https://borrow.nypl.org/search?query=The%20Hunger%20Games%20Suzanne%20Collins&searchType=everything&pageSize=10
Cover picture for row 0: None
Fetching URL: https://borrow.nypl.org/search?query=Harry%20Potter%20and%20the%20Order%20of%20the%20Phoenix%20J.K.%20Rowling,%20Mary%20GrandPré%20(Illustrator)&searchType=everything&pageSize=10
No img tag found for URL: https://borrow.nypl.org/search?query=Harry%20Potter%20and%20the%20Order%20of%20the%20Phoenix%20J.K.%20Rowling,%20Mary%20GrandPré%20(Illustrator)&searchType=everything&pageSize=10
Cover picture for row 1: None
Fetching URL: https://borrow.nypl.org/search?query=To%20Kill%20a%20Mockingbird%20Harper%20Lee&searchType=everything&pageSize=10
No img tag found for URL: https://borrow.nypl.org/search?query=To%20Kill%20a%20Mockingbird%20Harper%20Lee&searchType=everything&pageSize=10
Cover pict

KeyboardInterrupt: 

In [92]:
def fetch_cover_image_url(row):
    wordery_url = row['wordery_url']
    try:
        response = requests.get(wordery_url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        a_tag = soup.find('a', class_='c-book__media')
        # Extract the href attribute
        if a_tag:
            href_value = a_tag['href']
            return href_value
        else:
            print(f"No img tag found for URL: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None


for i, row in best_books.iterrows():
    cover_picture = fetch_cover_image_url(row)
    best_books.at[i, 'cover_picture'] = cover_picture
    print(f"Cover picture for row {i}: {cover_picture}")


Error fetching URL: 403 Client Error: Forbidden for url: https://wordery.com/search?term=The+Hunger+Games+Suzanne+Collins
Cover picture for row 0: None
Error fetching URL: 403 Client Error: Forbidden for url: https://wordery.com/search?term=Harry+Potter+and+the+Order+of+the+Phoenix+J.K.+Rowling,+Mary+GrandPr%C3%A9+(Illustrator)
Cover picture for row 1: None
Error fetching URL: 403 Client Error: Forbidden for url: https://wordery.com/search?term=To+Kill+a+Mockingbird+Harper+Lee
Cover picture for row 2: None
Error fetching URL: 403 Client Error: Forbidden for url: https://wordery.com/search?term=Pride+and+Prejudice+Jane+Austen,+Anna+Quindlen+(Introduction)
Cover picture for row 3: None
Error fetching URL: 403 Client Error: Forbidden for url: https://wordery.com/search?term=Twilight+Stephenie+Meyer
Cover picture for row 4: None
Error fetching URL: 403 Client Error: Forbidden for url: https://wordery.com/search?term=The+Book+Thief+Markus+Zusak+(Goodreads+Author)
Cover picture for row 5: No

KeyboardInterrupt: 

In [71]:

# def fetch_cover_image_url(row):
#     base_url = "https://borrow.nypl.org/search"
#     params = {
#         "query": row['searc_params_NYP'],
#         "searchType": "everything",
#         "pageSize": "1"
#     }
#     # headers={"User-Agent": "Mozilla/5.0"}
#     try:
#         response = requests.get(base_url, params=params)
#         response.raise_for_status()

#         soup = BeautifulSoup(response.text, 'html.parser')
#         img_tag = soup.find('img', class_='img-responsive img-thumbnail')
#         if img_tag:
#             return img_tag['src']
#         else:
#             print(f"No img tag found for URL: {response.status_code}")
#             # print(response.status_code)
#             return None

#     except requests.exceptions.RequestException as e:
#         print(f"Error fetching URL: {e}")
#         return None


# for i, row in best_books.iterrows():
#     cover_picture = get_cover(row)
#     best_books.at[i, 'cover_picture'] = cover_picture
#     print(f"Cover picture for row {i}: {cover_picture}")
#     if i % save_interval == 0:
#         best_books.to_csv(output_file, index=False)
#     time.sleep(2)


Fetching URL: https://borrow.nypl.org/search?query=The%20Hunger%20Games+Suzanne%20Collins&searchType=everything&pageSize=10
No img tag found for URL: https://borrow.nypl.org/search?query=The%20Hunger%20Games+Suzanne%20Collins&searchType=everything&pageSize=10
Cover picture for row 0: None
Fetching URL: https://borrow.nypl.org/search?query=Harry%20Potter%20and%20the%20Order%20of%20the%20Phoenix+J.K.%20Rowling,%20Mary%20GrandPré%20(Illustrator)&searchType=everything&pageSize=10
No img tag found for URL: https://borrow.nypl.org/search?query=Harry%20Potter%20and%20the%20Order%20of%20the%20Phoenix+J.K.%20Rowling,%20Mary%20GrandPré%20(Illustrator)&searchType=everything&pageSize=10
Cover picture for row 1: None
Fetching URL: https://borrow.nypl.org/search?query=To%20Kill%20a%20Mockingbird+Harper%20Lee&searchType=everything&pageSize=10
No img tag found for URL: https://borrow.nypl.org/search?query=To%20Kill%20a%20Mockingbird+Harper%20Lee&searchType=everything&pageSize=10
Cover picture for row 

KeyboardInterrupt: 

In [16]:
# # best_books['cover_picture'] =
# def get_cover(row):
#     url = row['url']
#     response = requests.get(url)
#     if response.status_code == 200:
#         soup = BeautifulSoup(response.content, 'html.parser')
#         image_tag = soup.select_one('.search-result-image-shadow')
#         if image_tag:
#             return image_tag.get('srcset')
#     else:
#         return None
# output_file = 'best_books_with_covers.csv'
# try:
#     existing_data = pd.read_csv(output_file)
#     processed_urls = set(existing_data['url'])
#     start_index = len(existing_data['url'])
#     best_books = pd.concat([existing_data, best_books[~best_books['url'].isin(processed_urls)]], ignore_index=True)
# except FileNotFoundError:
#     processed_urls = set()
#     existing_data = pd.DataFrame()
#     start_index = 0

# # Apply the get_cover function and save progress
# for i, row in best_books.iterrows():
#     if row['url'] in processed_urls:
#         continue
#     best_books.at[i, 'cover_picture'] = get_cover(row)
#     processed_urls.add(row['url'])

#     # Save every 10 rows or at the end
#     if (i + 1) % 10 == 0 or i == len(best_books) - 1:
#         best_books.to_csv(output_file, index=False)
#         print(f"Saved progress at row {i + 1}")

# best_books.to_csv(output_file, index=False)
# print("Completed scraping and saved final data.")

Saved progress at row 35760
Saved progress at row 35770
Saved progress at row 35780
Saved progress at row 35790
Saved progress at row 35800
Saved progress at row 35810
Saved progress at row 35820
Saved progress at row 35830
Saved progress at row 35840
Saved progress at row 35850
Saved progress at row 35860
Saved progress at row 35870
Saved progress at row 35880
Saved progress at row 35890
Saved progress at row 35900
Saved progress at row 35910
Saved progress at row 35920
Saved progress at row 35930
Saved progress at row 35940
Saved progress at row 35950
Saved progress at row 35960
Saved progress at row 35970
Saved progress at row 35980
Saved progress at row 35990
Saved progress at row 36000
Saved progress at row 36010
Saved progress at row 36020
Saved progress at row 36030
Saved progress at row 36040
Saved progress at row 36050
Saved progress at row 36060
Saved progress at row 36070
Saved progress at row 36080
Saved progress at row 36090
Saved progress at row 36100
Saved progress at ro

KeyboardInterrupt: 

In [296]:
best_books.head()

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,likedPercent,setting,coverImg,bbeScore,bbeVotes,price,bag_of_words,hyphenated_url_info,url,hyphenated_url_bookshop
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09,The Hunger Games Suzanne Collins ['Locus Award...,The-Hunger-Games/Suzanne-Collins/9780439023481,https://bookshop.org/search?keywords=The+Hunge...,The+Hunger+Games+Suzanne+Collins
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38,Harry Potter and the Order of the Phoenix J.K....,Harry-Potter-and-the-Order-of-the-Phoenix/J.K....,https://bookshop.org/search?keywords=Harry+Pot...,Harry+Potter+and+the+Order+of+the+Phoenix+J.K....
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN,To Kill a Mockingbird Harper Lee ['Pulitzer Pr...,To-Kill-a-Mockingbird/Harper-Lee/9999999999999,https://bookshop.org/search?keywords=To+Kill+a...,To+Kill+a+Mockingbird+Harper+Lee
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN,"Pride and Prejudice Jane Austen, Anna Quindlen...","Pride-and-Prejudice/Jane-Austen,-Anna-Quindlen...",https://bookshop.org/search?keywords=Pride+and...,"Pride+and+Prejudice+Jane+Austen,+Anna+Quindlen..."
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1,Twilight Stephenie Meyer ['Georgia Peach Book ...,Twilight/Stephenie-Meyer/9780316015844,https://bookshop.org/search?keywords=Twilight+...,Twilight+Stephenie+Meyer


In [282]:
# 1 - extract photo
# 2 - allow user to enter a book. it searches on the bookshop website for the book
# asks user to confirm which book they meant
# visits the page, extracts the description and runs the bag of words methods on it(tokenize, lemmetize, stop words)
# puts it in one of my 20 clusters, and then recommends books based on that.


,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price,bag_of_words,hyphenated_url_info
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09,The Hunger Games Suzanne Collins ['Locus Award...,The-Hunger-Games/Suzanne-Collins/9780439023481
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38,Harry Potter and the Order of the Phoenix J.K....,Harry-Potter-and-the-Order-of-the-Phoenix/J.K....
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN,To Kill a Mockingbird Harper Lee ['Pulitzer Pr...,To-Kill-a-Mockingbird/Harper-Lee/9999999999999
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN,"Pride and Prejudice Jane Austen, Anna Quindlen...","Pride-and-Prejudice/Jane-Austen,-Anna-Quindlen..."
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1,Twilight Stephenie Meyer ['Georgia Peach Book ...,Twilight/Stephenie-Meyer/9780316015844


In [9]:
best_books.head()

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,likedPercent,setting,coverImg,bbeScore,bbeVotes,price,bag_of_words,hyphenated_url_bookshop,url,wordery_url
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09,The Hunger Games Suzanne Collins ['Locus Award...,The+Hunger+Games+Suzanne+Collins,https://bookshop.org/search?keywords=The+Hunge...,https://wordery.com/search?term=The+Hunger+Gam...
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38,Harry Potter and the Order of the Phoenix J.K....,Harry+Potter+and+the+Order+of+the+Phoenix+J.K....,https://bookshop.org/search?keywords=Harry+Pot...,https://wordery.com/search?term=Harry+Potter+a...
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN,To Kill a Mockingbird Harper Lee ['Pulitzer Pr...,To+Kill+a+Mockingbird+Harper+Lee,https://bookshop.org/search?keywords=To+Kill+a...,https://wordery.com/search?term=To+Kill+a+Mock...
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN,"Pride and Prejudice Jane Austen, Anna Quindlen...","Pride+and+Prejudice+Jane+Austen,+Anna+Quindlen...",https://bookshop.org/search?keywords=Pride+and...,https://wordery.com/search?term=Pride+and+Prej...
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1,Twilight Stephenie Meyer ['Georgia Peach Book ...,Twilight+Stephenie+Meyer,https://bookshop.org/search?keywords=Twilight+...,https://wordery.com/search?term=Twilight+Steph...


In [10]:
best_books_for_BQ = best_books.drop(columns=['bbeVotes','bbeScore','bag_of_words', 'hyphenated_url_bookshop', 'url','wordery_url'])

In [20]:
best_books_for_BQ['book_id'] = range(1, len(best_books_for_BQ) + 1)


In [18]:

best_books_for_BQ = best_books_for_BQ.drop(columns=['bookId'])

In [22]:
best_books_for_BQ.head()

,title,series,author,rating,description,language,isbn,genres,characters,bookFormat,...,publishDate,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,price,book_id
0,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",Hardcover,...,09/14/08,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,5.09,1
1,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",Paperback,...,09/28/04,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,7.38,2
2,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",Paperback,...,05/23/06,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,NaN,3
3,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",Paperback,...,10/10/00,01/28/13,[],2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,NaN,4
4,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",Paperback,...,09/06/06,10/05/05,"['Georgia Peach Book Award (2007)', 'Buxtehude...",4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,2.1,5


In [23]:
best_books_for_BQ.to_csv('db_for_bq.csv', index=False)

In [24]:
from google.cloud import bigquery

In [26]:
best_books_for_BQ.to_gbq("Kaveri.books_data", project_id='da-bootcamp-2023', if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 3650.40it/s]


In [3]:
best_books.head()

NameError: name 'best_books' is not defined

In [30]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time
# import os

# # Function to save DataFrame to CSV
# def save_to_csv(dataframe, filename='book_covers.csv'):
#     if os.path.exists(filename):
#         dataframe.to_csv(filename, mode='a', header=False, index=False)
#     else:
#         dataframe.to_csv(filename, index=False)

# # Initialize an empty DataFrame to store all product information
# columns = ['wordery_url', 'book_cover']
# df = pd.DataFrame(columns=columns)

# # Initialize Chrome WebDriver
# driver = webdriver.Chrome()


# try:
#     for index, row in best_books.iterrows():
#         url = row['wordery_url']
#         print(f"Fetching URL: {url}")
#         driver.get(url)
#         time.sleep(10)  # Adding a delay to ensure the page loads completely

#         # Find elements with the specified class for book covers
#         book_cover = driver.find_element(By.CLASS_NAME, "c-book__media")

#         # Extract book cover URL
#         cover_url = book_cover.get_attribute('src')
#         df = pd.concat([df, pd.DataFrame({'wordery_url': [url], 'book_cover': [cover_url]})], ignore_index=True)
#         print(cover_url)

#         # Save the DataFrame to CSV after processing each URL
#         save_to_csv(df)
#         df = pd.DataFrame(columns=columns)  # Reset DataFrame to avoid duplicate entries

# finally:
#     # Quit the WebDriver
#     driver.quit()

Fetching URL: https://wordery.com/search?term=The+Hunger+Games+Suzanne+Collins
None
Fetching URL: https://wordery.com/search?term=Harry+Potter+and+the+Order+of+the+Phoenix+J.K.+Rowling,+Mary+GrandPré+(Illustrator)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".c-book__media"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010fcc40e8 chromedriver + 5169384
1   chromedriver                        0x000000010fcbbfba chromedriver + 5136314
2   chromedriver                        0x000000010f83836c chromedriver + 402284
3   chromedriver                        0x000000010f885740 chromedriver + 718656
4   chromedriver                        0x000000010f885a01 chromedriver + 719361
5   chromedriver                        0x000000010f8cabc4 chromedriver + 1002436
6   chromedriver                        0x000000010f8a8add chromedriver + 862941
7   chromedriver                        0x000000010f8c7f57 chromedriver + 991063
8   chromedriver     

In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

# Setup headless Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)
# Initialize Chrome WebDriver

# URL to scrape
book_pic_urls = []
for index, row in best_books.iterrows():
        url = row['wordery_url']
        print(f"Fetching URL: {url}")
        driver.get(url)
        time.sleep(10)
        book_cover = driver.find_element(By.CLASS_NAME, "c-book__media")
        book_pic_urls.append(book_cover.get_attribute('href'))
        print(book_cover.get_attribute('href'))
        driver.quit()
# List to store all product information


 # Adding a 2-second delay
# Find all elements with the specified class for product names





# Quit the WebDriver
driver.quit()

Fetching URL: https://wordery.com/search?term=The+Hunger+Games+Suzanne+Collins


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".c-book__media"}
  (Session info: chrome-headless-shell=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010695d0e8 chromedriver + 5169384
1   chromedriver                        0x0000000106954fba chromedriver + 5136314
2   chromedriver                        0x00000001064d136c chromedriver + 402284
3   chromedriver                        0x000000010651e740 chromedriver + 718656
4   chromedriver                        0x000000010651ea01 chromedriver + 719361
5   chromedriver                        0x0000000106563bc4 chromedriver + 1002436
6   chromedriver                        0x0000000106541add chromedriver + 862941
7   chromedriver                        0x0000000106560f57 chromedriver + 991063
8   ch

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

# Setup headless Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)


book_pic_urls = []

for url in urls:
    driver.get(url)

    try:
        # Wait until the book cover element is present
        book_cover = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "c-book__media"))
        )

        # Get the href attribute
        book_pic_urls.append(book_cover.get_attribute('href'))
        print(book_cover.get_attribute('href'))

    except Exception as e:
        print(f"Error fetching URL: {url}")
        print(str(e))

    # Random delay to mimic human behavior
    time.sleep(random.uniform(5, 10))

driver.quit()

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
# Initialize Chrome WebDriver
driver = webdriver.Chrome()
# URL to scrape
book_pic_urls = []
for index, row in best_books.iterrows():
        url = row['url']
        print(f"Fetching URL: {url}")
        driver.get(url)
        time.sleep(10)
        book_cover = driver.find_element(By.CLASS_NAME, "search-result-image-shadow")
        book_pic_urls.append(book_cover.get_attribute('srcset'))
        print(book_cover.get_attribute('srcset'))

# List to store all product information


 # Adding a 2-second delay
# Find all elements with the specified class for product names





# Quit the WebDriver
driver.quit()

Fetching URL: https://bookshop.org/search?keywords=The+Hunger+Games+Suzanne+Collins
None
Fetching URL: https://bookshop.org/search?keywords=Harry+Potter+and+the+Order+of+the+Phoenix+J.K.+Rowling,+Mary+GrandPré+(Illustrator)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".search-result-image-shadow"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010c5570e8 chromedriver + 5169384
1   chromedriver                        0x000000010c54efba chromedriver + 5136314
2   chromedriver                        0x000000010c0cb36c chromedriver + 402284
3   chromedriver                        0x000000010c118740 chromedriver + 718656
4   chromedriver                        0x000000010c118a01 chromedriver + 719361
5   chromedriver                        0x000000010c15dbc4 chromedriver + 1002436
6   chromedriver                        0x000000010c13badd chromedriver + 862941
7   chromedriver                        0x000000010c15af57 chromedriver + 991063
8   chro

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
# Initialize Chrome WebDriver
driver = webdriver.Chrome()
# URL to scrape
book_pic_urls = []
for index, row in best_books.iterrows():
        url = row['wordery_url']
        print(f"Fetching URL: {url}")
        driver.get(url)
        time.sleep(10)
        book_cover = driver.find_element(By.CLASS_NAME, "c-book__media")
        book_pic_urls.append(book_cover.get_attribute('href'))
        print(book_cover.get_attribute('href'))
        driver.quit()
# List to store all product information


 # Adding a 2-second delay
# Find all elements with the specified class for product names





# Quit the WebDriver


Fetching URL: https://wordery.com/search?term=The+Hunger+Games+Suzanne+Collins
https://wordery.com/the-ballad-of-songbirds-snakes-suzanne-collins-9780702328909?cTrk=MjA0OTUzMzgyfDY2OGM0YWYxMTQyNjA6MToxOjY2OGM0YWYxMDFiYmE0LjM2NTY1MzQ5OmY1ZjZjNzk5
Fetching URL: https://wordery.com/search?term=Harry+Potter+and+the+Order+of+the+Phoenix+J.K.+Rowling,+Mary+GrandPré+(Illustrator)


MaxRetryError: HTTPConnectionPool(host='localhost', port=54131): Max retries exceeded with url: /session/a28e6916c9c515ddd65f44b923711df4/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x150268a10>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import random
import time



# Setup headless Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

# List to store book pic URLs
book_pic_urls = []

try:
    for index, row in best_books.iterrows():
        url = row['wordery_url']
        print(f"Fetching URL: {url}")

        driver.get(url)

        # Explicit wait for the book cover element
        try:
            book_cover = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "c-book__media"))
            )
            book_pic_urls.append(book_cover.get_attribute('href'))
            print(book_cover.get_attribute('href'))

        except Exception as e:
            print(f"Error fetching URL: {url}")
            print(str(e))

        # Random delay to mimic human behavior
        time.sleep(random.uniform(5, 10))

finally:
    driver.quit()

# Displaying the collected URLs
print("Book Picture URLs:")
for url in book_pic_urls:
    print(url)

Fetching URL: https://wordery.com/search?term=The+Hunger+Games+Suzanne+Collins
Error fetching URL: https://wordery.com/search?term=The+Hunger+Games+Suzanne+Collins
Message: 
Stacktrace:
0   chromedriver                        0x000000010d5890e8 chromedriver + 5169384
1   chromedriver                        0x000000010d580fba chromedriver + 5136314
2   chromedriver                        0x000000010d0fd36c chromedriver + 402284
3   chromedriver                        0x000000010d14a740 chromedriver + 718656
4   chromedriver                        0x000000010d14aa01 chromedriver + 719361
5   chromedriver                        0x000000010d18fbc4 chromedriver + 1002436
6   chromedriver                        0x000000010d16dadd chromedriver + 862941
7   chromedriver                        0x000000010d18cf57 chromedriver + 991063
8   chromedriver                        0x000000010d16d853 chromedriver + 862291
9   chromedriver                        0x000000010d13d5c6 chromedriver + 665030
1

KeyboardInterrupt: 

In [7]:
# bookcover
# https://openlibrary.org/search?q=twilight+stephanie+meyer&mode=everything
best_books['open_library_url']= "https://openlibrary.org/search?q="+best_books['title'].str.split(" ").str.join("+")+"+"+best_books["author"].str.split(" ").str.join("+")+"&mode=everything"

In [9]:
def get_cover(row):
    url = row['open_library_url']
    print(f"Fetching URL: {url}")
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            span_bookcover = soup.find('span', class_='bookcover')
            if span_bookcover:
                img_element = span_bookcover.find('img')
                if img_element:
                    src_value = img_element.get('src')
                    print("SRC Attribute:", src_value)
                    return src_value
                else:
                    print("No img tag found inside span with class 'bookcover'")
                    return None
            else:
                print("No span found with class 'bookcover'")
                return None
        else:
            print(f"Failed to fetch URL: {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception occurred while fetching URL {url}: {e}")
        return None



output_file = 'best_books_with_covers.csv'
save_interval = 10  # Save progress every 10 iterations

for i, row in best_books.iterrows():
    best_books.at[i, 'cover_picture'] = get_cover(row)
    print(f"Cover picture for row {i}: {best_books.at[i, 'cover_picture']}")
    if i % save_interval == 0:
        best_books.to_csv(output_file, index=False)
    time.sleep(2)  # Delay between requests to avoid being blocked

# Final save
best_books.to_csv(output_file, index=False)

# Check the DataFrame
print(best_books)

Fetching URL: https://openlibrary.org/search?q=The+Hunger+Games+Suzanne+Collins&mode=everything
SRC Attribute: //covers.openlibrary.org/b/id/13575602-M.jpg
Cover picture for row 0: //covers.openlibrary.org/b/id/13575602-M.jpg
Fetching URL: https://openlibrary.org/search?q=Harry+Potter+and+the+Order+of+the+Phoenix+J.K.+Rowling,+Mary+GrandPré+(Illustrator)&mode=everything
SRC Attribute: //covers.openlibrary.org/b/id/11416565-M.jpg
Cover picture for row 1: //covers.openlibrary.org/b/id/11416565-M.jpg
Fetching URL: https://openlibrary.org/search?q=To+Kill+a+Mockingbird+Harper+Lee&mode=everything
SRC Attribute: //covers.openlibrary.org/b/id/12784310-M.jpg
Cover picture for row 2: //covers.openlibrary.org/b/id/12784310-M.jpg
Fetching URL: https://openlibrary.org/search?q=Pride+and+Prejudice+Jane+Austen,+Anna+Quindlen+(Introduction)&mode=everything
SRC Attribute: //covers.openlibrary.org/b/id/12818799-M.jpg
Cover picture for row 3: //covers.openlibrary.org/b/id/12818799-M.jpg
Fetching URL: ht

KeyboardInterrupt: 

In [12]:
best_books['coverImg'][2]

'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1553383690l/2657.jpg'